### kaggle playground private 2등
* 간단한 모형 구현연습하기에 playground는 좋은 것 같다

In [1]:
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import time
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import torch
from torch import nn
from torch import tensor
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

import functions
import Linear

In [2]:
train = pd.read_csv('../copy-of-forecasting-mini-course-sales/train.csv')
test = pd.read_csv('../copy-of-forecasting-mini-course-sales/test.csv')
train.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [3]:
train['date']=pd.to_datetime(train['date'])
test['date']=pd.to_datetime(test['date'])

In [4]:
pd.Series([x.year for x in train['date'].unique()]).value_counts()

2020    366
2017    365
2018    365
2019    365
2021    365
Name: count, dtype: int64

In [5]:
#train = train.loc[(train['date'] != '2020-02-29')] # 큰 의미는 없네

In [6]:
label_encoders = {}
column_list = ['country','store','product']
for c in column_list:
    label_encoders[c] = LabelEncoder()
    label_encoders[c].fit(train[c])
    
    train[c] = label_encoders[c].transform(train[c])
    test[c] = label_encoders[c].transform(test[c])

In [7]:
def data_process(x, yrain_years):
    x = x.sort_values('date')['num_sold'].values.tolist()
    window_size = 365
    full_size = window_size*(train_years+1)
    train_size = window_size*train_years
    
    trey = []
    for i in range(len(x)-full_size+1):
        trey.append([x[i:i+train_size], x[i+train_size:i+full_size]])
    return trey
train_years = 3
reshaped = pd.concat([train, test]).groupby(['country', 'store', 'product']).apply(lambda x : data_process(x[['date','num_sold']], train_years))

In [8]:
#test : reshaped[0][0][0][-1][0]
# test : reshaped[0][0][0][-366][0]

In [9]:
feature = []
target = []
test_feature = []
test_target = []
for values in tqdm(reshaped.values):
    for value in values[:-365]:
        feature.append(value[0])
        target.append(value[1])
        
    test_feature.append(values[-1][0])
    test_target.append(values[-1][1])
feature = pd.DataFrame(feature)
target = pd.DataFrame(target)
test_feature = tensor(test_feature)
test_target = tensor(test_target)

100%|███████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 12495.94it/s]


In [10]:
train_feature = feature.sample(int(feature.shape[0]*0.9), random_state = 1004)
idx = train_feature.index
train_feature = tensor(train_feature.values)
train_target = tensor(target.loc[idx].values)

valid_feature = tensor(feature.drop(idx).values)
valid_target = tensor(target.drop(idx).values)

In [11]:
train_loader = DataLoader(TensorDataset(train_feature, train_target), batch_size = 2**10, shuffle = True)
valid_loader = DataLoader(TensorDataset(valid_feature, valid_target), batch_size = 2**25, shuffle = False)

In [12]:
torch.manual_seed(1004)

n_linear = Linear.NLinear(365*train_years,365)

optimizer = AdamW(n_linear.parameters(), lr = 1e-2, eps = 1e-16)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor = 0.5, patience=0, verbose = True)
start = time.time()
best_loss = 0
early_count = 0
epoch = 0
while((early_count < 5) & (time.time()-start < 1000)):
    epoch += 1
    losses = []
    nums = []
    for batch in train_loader:
        last = batch[0][:,-1].clone().unsqueeze(1)
        batch[0] /= last
        batch[1] /= last
        logits = n_linear(batch[0].float())
        
        loss, num = functions.get_loss(logits, batch[1].float(), nn.MSELoss(), optimizer)
        losses.append(loss)
        nums.append(num)
    train_loss = np.sum(np.multiply(losses, nums))/np.sum(nums)

    losses = []
    nums = []
    for batch in valid_loader:
        last = batch[0][:,-1].clone().unsqueeze(1)
        batch[0] /= last
        batch[1] /= last
        logits = n_linear(batch[0].float())
        
        loss, num = functions.get_loss(logits, batch[1].float(), nn.MSELoss(), optimizer)
        losses.append(loss)
        nums.append(num)
    valid_loss = np.sum(np.multiply(losses, nums))/np.sum(nums)
    scheduler.step(valid_loss)
    
    print('train loss : ', train_loss, ' & valid loss : ', valid_loss, ' & epoch : ', epoch, ' & time', time.time()-start)
    
    if best_loss == 0:
        best_loss = valid_loss
        torch.save(n_linear, '../copy-of-forecasting-mini-course-sales/NLinear.pt')
        print('best model save at NLinear.pt')
    else:
        if best_loss > valid_loss:
            best_loss = valid_loss
            torch.save(n_linear, '../copy-of-forecasting-mini-course-sales/NLinear.pt')
            print('best model save at NLinear.pt')
            early_count = 0
        else:
            early_count += 1

train loss :  1.2098925922859394  & valid loss :  0.34467998147010803  & epoch :  1  & time 1.3909635543823242
best model save at NLinear.pt
train loss :  0.1163167437123748  & valid loss :  0.05337835103273392  & epoch :  2  & time 2.787266254425049
best model save at NLinear.pt
train loss :  0.04073857112542389  & valid loss :  0.03413711115717888  & epoch :  3  & time 4.155574560165405
best model save at NLinear.pt
train loss :  0.0312984124654337  & valid loss :  0.029325103387236595  & epoch :  4  & time 6.196982383728027
best model save at NLinear.pt
train loss :  0.028353015791601325  & valid loss :  0.02718854509294033  & epoch :  5  & time 7.604169845581055
best model save at NLinear.pt
train loss :  0.026742050410382932  & valid loss :  0.025927435606718063  & epoch :  6  & time 9.003443002700806
best model save at NLinear.pt
train loss :  0.025320426479881863  & valid loss :  0.024746853858232498  & epoch :  7  & time 10.398537635803223
best model save at NLinear.pt
train lo

In [13]:
forsub = test.sort_values(['country','store','product', 'date'])

In [14]:
last = test_feature[:,-1].clone().unsqueeze(1)
test_feature_scaled = test_feature / last ## scaling을 진행하지 않는 것이 좋게 나온다. 왜지?

In [15]:
n_linear = torch.load('../copy-of-forecasting-mini-course-sales/NLinear.pt')
forsub['num_sold'] = [abs(x) for x in n_linear(test_feature.float()).reshape(-1).tolist()]
forsub

,id,date,country,store,product,num_sold
10,136960,2022-01-01,0,0,0,239.380264
85,137035,2022-01-02,0,0,0,248.390228
160,137110,2022-01-03,0,0,0,211.368896
235,137185,2022-01-04,0,0,0,212.097107
310,137260,2022-01-05,0,0,0,205.316177
...,...,...,...,...,...,...
27069,164019,2022-12-27,4,2,4,88.817276
27144,164094,2022-12-28,4,2,4,98.842087
27219,164169,2022-12-29,4,2,4,109.833572
27294,164244,2022-12-30,4,2,4,119.414207


In [16]:
submissions = pd.read_csv('../copy-of-forecasting-mini-course-sales/sample_submission.csv')

In [17]:
submissions.index = submissions.id
forsub.index = forsub.id
pd.concat([submissions[['id']], forsub[['num_sold']]], axis=1).to_csv('../copy-of-forecasting-mini-course-sales/submission.csv', index=False)